In [7]:
import requests
import pandas as pd
import re
from datetime import date

## reminder: beta version was saved before I took out all the testing cells

## 10/30 This version adds map stats and "date queried"

In [2]:
def check_date(date):
    try:
        date_pattern = r'\d{4}-\d{2}-\d{2}'
        if re.match(date_pattern, date):
            if int(date[:4]) < 2019:
                return f'after={date}T12%3A02%3A36.874Z'
            else:
                return f'before={date}T12%3A02%3A36.874Z'
    except:
        return ''

In [3]:
query_date = input('enter YYYY-MM-DD for a specific date. improper date formats will show most recent songs') ## This will turn into a button on streamlit

bs_url = 'https://api.beatsaver.com/maps/latest?' 
search_date_formatted = check_date(query_date)
url_end = '&pageSize=50'


In [4]:
# gets fresh data from beat saver
response = requests.get(f'{bs_url}{search_date_formatted}{url_end}').json()

# looking at the guts

In [5]:
response['docs'][0] #ok, saving one cell that looks at the first song

{'id': '31ff7',
 'name': 'Pirendora - Midnight Factory',
 'description': "🌙for rank🌟\n\n🌉Original🌃\nhttps://soundcloud.com/pirendora/midnight-factory\n\nIt just so happens that I was able to upload it on my birthday.\nThis map will be one of the most important maps for me!\n\nTest : @yas  @yaomin\nRT MOD: @ThySpoon, @yabje\nYou have improved this map for the better, thank you!\n\n2023/06/13 Re Publish!\nRT MOD received.\n\n2023/07/16 Re Publish!\nRT MOD received. Correction of minor details.\n\n2023/07/26\nI'm now Qualified!\n\n2023/08/09\nRank!",
 'uploader': {'id': 4284625,
  'name': 'zeon-gitien',
  'hash': '6066e91fb85f2d00062d465f',
  'avatar': 'https://cdn.beatsaver.com/avatar/b2fd3baa5620c310a7624475e11e83ea0a5ba1ed.png',
  'type': 'DISCORD',
  'admin': False,
  'curator': False,
  'verifiedMapper': True,
  'playlistUrl': 'https://api.beatsaver.com/users/id/4284625/playlist'},
 'metadata': {'bpm': 153.0,
  'duration': 144,
  'songName': 'Midnight Factory',
  'songSubName': '',
 

In [ ]:
response['docs'][i]['stats']['upvotes']
response['docs'][i]['stats']['downvotes']
response['docs'][i]['stats']['score']

In [9]:
now = date.today()

print(now)

2023-10-30


# functional stuff

In [99]:
bs_data = []

In [100]:
# makes a dictionary from the data of each song in response, then adds it to the BS data list.

for i in range(len(response['docs'])):
    song = {}
    song['bs_map_id'] = response['docs'][i]['id']
    song['title'] = response['docs'][i]['metadata']['songName']
    song['artist'] = response['docs'][i]['metadata']['songAuthorName']
    song['mapper_name'] = response['docs'][i]['uploader']['name']
    song['mapper_id'] = response['docs'][i]['uploader']['id']
    song['duration_seconds'] = response['docs'][i]['metadata']['duration']
    song['auto_mapped'] = response['docs'][i]['automapper']
    song['upload_date'] = response['docs'][i]['lastPublishedAt']
    try: 
        song['tags'] = response['docs'][i]['tags']
    except:
        song['tags'] = ['none']
    song['difficulties'] = [response['docs'][i]['versions'][0]['diffs'][x]['difficulty'] for x in range(len(response['docs'][i]['versions'][0]['diffs']))]
    bs_data.append(song)


In [102]:
#turn the list into a dataframe

df = pd.DataFrame(bs_data)
df.shape

(50, 10)

In [103]:
# Normal Export, will overwrite existing data every time:
df.to_csv('beatsaver_songs.csv') 

# append export (depreciated):
# df.to_csv('beatsaver_songs.csv', mode='a', headers=False)